In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import wrds
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats

from get_stock_data import GetStockData

In [3]:
username = "mth989722"
password = "SystTrade2022"
conn = wrds.Connection(wrds_username='mth989722')

Loading library list...
Done


In [22]:
# conn.describe_table(library='crsp', table='dsf')
# conn.describe_table(library='crsp', table='msenames')

In [5]:
for ticker in ['PNR', 'CEQP', 'PAY', 'AKCA', 'SJW', 'CRM', 'WHG', 'MNKPF', 'S',
       'SFTBY', 'NFLX', 'TWNK', 'REED', 'ILMN', 'SXCP', 'IMAX', 'CXO',
       'TNRG', 'BH', 'WEC', 'CTXS', 'CYTK', 'ALXN', 'AKS', 'STAG', 'HUBG',
       'QLYS', 'EFII', 'HROW', 'ACUR', 'NNUTU', 'CCI', 'ACAD', 'PYPL',
       'VNO', 'WTRG', 'ITRI', 'ORBC', 'GDDY']:
    try:
        data = GetStockData._get_stock_data(conn=conn, ticker=ticker, startdate='01/01/2018', enddate='12/31/2021')
        if not data is None:
            data.to_csv(f"./stock_data/{ticker}.csv")
    except Exception as e: 
        print(ticker)
        raise e

Empty DataFrame
Columns: [permno, permco, date, shrcd, exchcd, ticker, ret, retx, shrout, prc, cfacshr, openprc, vol, askhi, bidlo]
Index: []
Empty DataFrame
Columns: [permno, permco, date, shrcd, exchcd, ticker, ret, retx, shrout, prc, cfacshr, openprc, vol, askhi, bidlo]
Index: []
Empty DataFrame
Columns: [permno, permco, date, shrcd, exchcd, ticker, ret, retx, shrout, prc, cfacshr, openprc, vol, askhi, bidlo]
Index: []
Empty DataFrame
Columns: [permno, permco, date, shrcd, exchcd, ticker, ret, retx, shrout, prc, cfacshr, openprc, vol, askhi, bidlo]
Index: []
Empty DataFrame
Columns: [permno, permco, date, shrcd, exchcd, ticker, ret, retx, shrout, prc, cfacshr, openprc, vol, askhi, bidlo]
Index: []


In [10]:
data = pd.read_csv('./edgar_data/2018-01-04.csv')
data.ticker.unique()

array(['PNR', 'CEQP', 'PAY', 'AKCA', 'SJW', 'CRM', 'WHG', 'MNKPF', 'S',
       'SFTBY', 'NFLX', 'TWNK', 'REED', 'ILMN', 'SXCP', 'IMAX', 'CXO',
       'TNRG', 'BH', 'WEC', 'CTXS', 'CYTK', 'ALXN', 'AKS', 'STAG', 'HUBG',
       'QLYS', 'EFII', 'HROW', 'ACUR', 'NNUTU', 'CCI', 'ACAD', 'PYPL',
       'VNO', 'WTRG', 'ITRI', 'ORBC', 'GDDY'], dtype=object)

In [4]:
data = GetStockData._get_stock_data(conn=conn, ticker='MNKPF', startdate='01/01/2018', enddate='12/31/2021')

Empty DataFrame
Columns: [permno, permco, date, shrcd, exchcd, ticker, ret, retx, shrout, prc, cfacshr, openprc, vol, askhi, bidlo]
Index: []
